In [56]:
import re
import os
import numpy as np
import cPickle
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator

class MajorityVoteClassifier(BaseEstimator,
                             ClassifierMixin):
    """ A majority vote ensemble classifier
    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble
    vote : str, {'classlabel', 'probability'} (default='label')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).
    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.
    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.
        y : array-like, shape = [n_samples]
            Vector of target class labels.
        Returns
        -------
        self : object
        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.
        Returns
        ----------
        maj_vote : array-like, shape = [n_samples]
            Predicted class labels.
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                lambda x:
                np.argmax(np.bincount(x,
                                      weights=self.weights)),
                axis=1,
                arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg_proba : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
        return out



with open('model.pkl', 'rb') as fid:
    pipe = cPickle.load(fid)

In [59]:
def creatematrix(features, kmer):
    stat = pd.read_csv(features, sep=",")
    kmer = pd.read_csv(kmer, sep="\t", header = None)
    kmer = kmer.iloc[:, :-1]
    id2 = stat.id.str.split("-",expand=True) # split the string to get label
    id2 = id2.iloc[:, :-1]
    stat2 = stat.iloc[:, 1:]
    df = pd.concat([stat2.reset_index(drop=True), kmer], axis=1) # concat kmerand stat matrix
    df = pd.concat([id2, df], axis=1)
    df.columns.values[0] = "label"
    # encoding class labels as integers
    df.loc[df.label == 'positive', 'label'] = 1
    df.loc[df.label == 'negative', 'label'] = 0
    return df

dat = createpredictmatrix('dat/input.features.clear2.csv', 'dat/input.features.kmer')
X = dat.drop(dat.columns[[0]], 1) # remove label

In [60]:
X

,length,ncodons,nUnk,nG,nA,nT,nC,nN,nGC,nAT,...,638,639,640,641,642,643,644,645,646,647
0,420,140,0,87,76,134,123,0,210,210,...,0,0.002381,0.000000,0.004762,0.000000,0.002381,0.002381,0.002381,0.000000,0.000000
1,490,163,0,104,86,154,146,0,250,240,...,0,0.004082,0.000000,0.004082,0.000000,0.002041,0.002041,0.004082,0.000000,0.000000
2,490,163,0,109,140,139,102,0,211,279,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002041,0.000000
3,140,46,0,33,37,45,25,0,58,82,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,210,70,0,47,41,70,52,0,99,111,...,0,0.000000,0.000000,0.004762,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,420,140,0,113,112,92,103,0,216,204,...,0,0.000000,0.000000,0.007143,0.000000,0.004762,0.004762,0.002381,0.002381,0.002381
6,420,140,0,84,117,120,99,0,183,237,...,0,0.002381,0.002381,0.007143,0.000000,0.000000,0.004762,0.000000,0.004762,0.000000
7,420,140,0,91,114,114,101,0,192,228,...,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.002381,0.000000,0.004762,0.000000
8,700,233,0,169,183,195,153,0,322,378,...,0,0.000000,0.000000,0.000000,0.001429,0.000000,0.001429,0.000000,0.005714,0.002857
9,560,186,0,104,183,167,106,0,210,350,...,0,0.000000,0.001786,0.000000,0.000000,0.001786,0.000000,0.001786,0.000000,0.000000


In [66]:
for index, row in X.iterrows():


0
1
2
3
4
5
6
7
8
9


In [64]:
label = {0:'chromosomal', 1:'plasmid'}


print('Prediction: %s\nProbability: %.2f%%' %\
      (label[pipe.predict(X)[6]], pipe.predict_proba(X)[6].max()*100))

#pipe.predict_proba(X_test)[0].max()

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    0.2s


Prediction: plasmid
Probability: 64.84%


[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished


In [ ]:
X